In [1]:
#data preprocessing
import os
from PIL import Image, ImageEnhance
import numpy as np

def load_and_preprocess_images(directory, target_size=(128, 128)):
    images = []
    labels = []  # This could be modified to parse actual labels if available

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                with Image.open(filepath) as img:
                    # Enhance image contrast
                    enhancer = ImageEnhance.Contrast(img)
                    img = enhancer.enhance(2.0)  # Enhance contrast; you can adjust the factor as needed

                    # Resize and convert to grayscale
                    img = img.resize(target_size).convert('L')

                    # Convert to numpy array and normalize
                    img_array = np.array(img) / 255.0
                    images.append(img_array)
                    # Extract labels from filename if applicable
                    # labels.append(extract_label_from_filename(filename))
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

    # Convert list of arrays to a 4D NumPy array: (num_samples, height, width, channels)
    images = np.array(images)[..., np.newaxis]  # Adding channel dimension for grayscale
    return images, labels

# Usage
directory = '/Users/laila_1/Downloads/SOCOFing/Real'  # Update this path to where your dataset is stored
images, labels = load_and_preprocess_images(directory)

print("Shape of the images array:", images.shape)  # Should be (num_samples, 128, 128, 1)
print("Number of labels:", len(labels))  # Should match num_samples if labels are processed


Shape of the images array: (6000, 128, 128, 1)
Number of labels: 0


In [5]:
import numpy as np
#data preprocessing
import os
from PIL import Image, ImageEnhance
import numpy as np

def load_and_preprocess_images(directory, target_size=(128, 128)):
    images = []
    labels = []  # This could be modified to parse actual labels if available

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                with Image.open(filepath) as img:
                    # Enhance image contrast
                    enhancer = ImageEnhance.Contrast(img)
                    img = enhancer.enhance(2.0)  # Enhance contrast; you can adjust the factor as needed

                    # Resize and convert to grayscale
                    img = img.resize(target_size).convert('L')

                    # Convert to numpy array and normalize
                    img_array = np.array(img) / 255.0
                    images.append(img_array)
                    # Extract labels from filename if applicable
                    # labels.append(extract_label_from_filename(filename))
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

    # Convert list of arrays to a 4D NumPy array: (num_samples, height, width, channels)
    images = np.array(images)[..., np.newaxis]  # Adding channel dimension for grayscale
    return images, labels

# Usage
directory = '/Users/laila_1/Downloads/SOCOFing/Real'  # Update this path to where your dataset is stored
images, labels = load_and_preprocess_images(directory)


class ConvNet:
    def __init__(self, image):
        # Hyperparameters
        self.num_filters = 8  # Increased number of filters
        self.filter_h = 3
        self.filter_w = 3
        self.stride = 1

        # Weights initialization
        self.filters_weights = np.random.randn(self.num_filters, self.filter_h, self.filter_w)
        self.bias = np.random.randn(self.num_filters, 1, 1)

        # Image dimensions
        self.examples = image.shape[0]
        self.img_height = image.shape[1]
        self.img_width = image.shape[2]

        # Keep original input for later
        self.last_input = image

    # Activation function
    def relu(self, x):
        return np.maximum(0.0, x)

    # Padding function
    def zero_pad(self, image_to_pad, pad_h, pad_w):
        return np.pad(image_to_pad, ((0, 0), (pad_h, pad_h), (pad_w, pad_w), (0, 0)), 'constant', constant_values=(0, 0))

    # Convolution forward propagation
    def conv_forward(self, padding_mode):
        if padding_mode == "same":
            pad_h = (self.filter_h - 1) // 2
            pad_w = (self.filter_w - 1) // 2
            self.last_input = self.zero_pad(self.last_input, pad_h, pad_w)
            self.img_height += 2 * pad_h
            self.img_width += 2 * pad_w

        self.fm_height = (self.img_height - self.filter_h) // self.stride + 1
        self.fm_width = (self.img_width - self.filter_w) // self.stride + 1
        self.f_map = np.zeros((self.examples, self.fm_height, self.fm_width, self.num_filters))

        for n in range(self.examples):
            for i in range(self.fm_height):
                for j in range(self.fm_width):
                    for f in range(self.num_filters):
                        row_start = i * self.stride
                        col_start = j * self.stride
                        local_region = self.last_input[n, row_start:row_start + self.filter_h, col_start:col_start + self.filter_w, :]
                        self.f_map[n, i, j, f] = np.sum(local_region * self.filters_weights[f]) + self.bias[f]

        return self.relu(self.f_map)

# Example usage
# Assuming `image` is your input image data as a NumPy array of shape (num_examples, height, width, channels)
# Initialize and run the convolution
convnet = ConvNet(images)
feature_map = convnet.conv_forward('same')
print(feature_map.shape)
print(feature_map)

# Continue with MaxPooling, Flatten, and Neural Network classes with similar revisions...
class MaxPooling:
    def __init__(self, f_map, pool_h=2, pool_w=2, stride=2):
        self.pool_filter_h = pool_h
        self.pool_filter_w = pool_w
        self.stride = stride

        self.examples = f_map.shape[0]
        self.fm_height = f_map.shape[1]
        self.fm_width = f_map.shape[2]
        self.channels = f_map.shape[3]
        self.last_input = f_map

    def max_forward(self):
        height_out = (self.fm_height - self.pool_filter_h) // self.stride + 1
        width_out = (self.fm_width - self.pool_filter_w) // self.stride + 1
        self.pool_output = np.zeros((self.examples, height_out, width_out, self.channels))

        for n in range(self.examples):
            for c in range(self.channels):
                for i in range(height_out):
                    for j in range(width_out):
                        row_start = i * self.stride
                        col_start = j * self.stride
                        window = self.last_input[n, row_start:row_start + self.pool_filter_h, col_start:col_start + self.pool_filter_w, c]
                        self.pool_output[n, i, j, c] = np.max(window)

        return self.pool_output


def flatten_layer(f_map):
    flat_output = f_map.reshape(f_map.shape[0], -1)
    return flat_output


class NeuralNetwork:
    def __init__(self, input_size, num_classes, hidden_units=100, learning_rate=0.01):
        self.learning_rate = learning_rate

        # Initialize weights
        self.weights1 = np.random.randn(input_size, hidden_units) * 0.01
        self.bias1 = np.zeros((1, hidden_units))
        self.weights2 = np.random.randn(hidden_units, num_classes) * 0.01
        self.bias2 = np.zeros((1, num_classes))

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)

    def forward(self, X):
        self.Z1 = np.dot(X, self.weights1) + self.bias1
        self.A1 = self.sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.weights2) + self.bias2
        self.A2 = self.softmax(self.Z2)
        return self.A2

    def compute_loss(self, Y, Y_hat):
        m = Y.shape[0]
        log_probs = -np.log(Y_hat[range(m), Y])
        loss = np.sum(log_probs) / m
        return loss

    def backprop(self, X, Y, Y_hat):
        m = X.shape[0]
        
        # Gradient of loss w.r.t. output
        dZ2 = Y_hat
        dZ2[range(m), Y] -= 1
        dW2 = np.dot(self.A1.T, dZ2) / m
        dB2 = np.sum(dZ2, axis=0, keepdims=True) / m
        
        dA1 = np.dot(dZ2, self.weights2.T)
        dZ1 = dA1 * self.A1 * (1 - self.A1)  # Sigmoid derivative
        dW1 = np.dot(X.T, dZ1) / m
        dB1 = np.sum(dZ1, axis=0, keepdims=True) / m
        
        # Update weights and biases
        self.weights1 -= self.learning_rate * dW1
        self.bias1 -= self.learning_rate * dB1
        self.weights2 -= self.learning_rate * dW2
        self.bias2 -= self.learning_rate * dB2


# Assuming you have labels `y` that are integer class labels
nn = NeuralNetwork(flattened_output.shape[1], num_classes=10)  # num_classes depends on your dataset
predictions = nn.forward(flattened_output)
loss = nn.compute_loss(y, predictions)
nn.backprop(flattened_output, y, predictions)


KeyboardInterrupt: 

In [3]:
# Assuming you have a function to load and preprocess images
images, labels = load_and_preprocess_images('/Users/laila_1/Downloads/SOCOFing/Real')

# Assuming the preprocessing function returns images in the correct format
convnet = ConvNet(images)
feature_map = convnet.conv_forward('same')
print("Feature map shape:", feature_map.shape)
print("Feature map output:", feature_map)

# Proceed with max pooling, flattening, and feeding into a neural network
max_pooling = MaxPooling(feature_map)
pooled_feature_map = max_pooling.max_forward()
flattened_output = flatten_layer(pooled_feature_map)

# Assuming you have labels formatted correctly for your dataset
nn = NeuralNetwork(flattened_output.shape[1], num_classes=10)  # Adjust num_classes based on your actual dataset
predictions = nn.forward(flattened_output)
loss = nn.compute_loss(labels, predictions)
nn.backprop(flattened_output, labels, predictions)


KeyboardInterrupt: 

In [ ]:
import numpy as np
import os
from PIL import Image, ImageEnhance

def load_and_preprocess_images(directory, target_size=(128, 128)):
    images = []
    labels = []  # This could be modified to parse actual labels if available

    # Iterate over files in the directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                with Image.open(filepath) as img:
                    # Enhance image contrast
                    enhancer = ImageEnhance.Contrast(img)
                    img = enhancer.enhance(2.0)  # Enhance contrast; you can adjust the factor as needed

                    # Resize and convert to grayscale
                    img = img.resize(target_size).convert('L')

                    # Convert to numpy array and normalize
                    img_array = np.array(img) / 255.0
                    images.append(img_array)
                    # Extract labels from filename if applicable
                    # labels.append(extract_label_from_filename(filename))
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

    # Convert list of arrays to a 4D NumPy array: (num_samples, height, width, channels)
    images = np.array(images)[..., np.newaxis]  # Adding channel dimension for grayscale
    return images, labels

def batch_process(images, batch_size=100):
    num_batches = len(images) // batch_size + (1 if len(images) % batch_size != 0 else 0)
    for i in range(num_batches):
        batch_images = images[i * batch_size:(i + 1) * batch_size]
        yield batch_images

# Initialize ConvNet
class ConvNet:
    def __init__(self, num_filters=8, filter_h=3, filter_w=3, stride=1):
        self.num_filters = num_filters
        self.filter_h = filter_h
        self.filter_w = filter_w
        self.stride = stride
        self.filters_weights = np.random.randn(self.num_filters, self.filter_h, self.filter_w)
        self.bias = np.random.randn(self.num_filters, 1, 1)

    def relu(self, x):
        return np.maximum(0.0, x)

    def zero_pad(self, image_to_pad, pad_h, pad_w):
        return np.pad(image_to_pad, ((0, 0), (pad_h, pad_h), (pad_w, pad_w), (0, 0)), 'constant', constant_values=(0, 0))

    def conv_forward(self, image, padding_mode):
        if padding_mode == "same":
            pad_h = (self.filter_h - 1) // 2
            pad_w = (self.filter_w - 1) // 2
            image = self.zero_pad(image, pad_h, pad_w)
        img_height = image.shape[1]
        img_width = image.shape[2]
        fm_height = (img_height - self.filter_h) // self.stride + 1
        fm_width = (img_width - self.filter_w) // self.stride + 1
        f_map = np.zeros((image.shape[0], fm_height, fm_width, self.num_filters))
        for n in range(image.shape[0]):
            for i in range(fm_height):
                for j in range(fm_width):
                    for f in range(self.num_filters):
                        row_start = i * self.stride
                        col_start = j * self.stride
                        local_region = image[n, row_start:row_start + self.filter_h, col_start:col_start + self.filter_w, :]
                        f_map[n, i, j, f] = np.sum(local_region * self.filters_weights[f]) + self.bias[f]
        return self.relu(f_map)

directory = '/Users/laila_1/Downloads/SOCOFing/Real'  # Update this path to where your dataset is stored
images, labels = load_and_preprocess_images(directory)
convnet = ConvNet()

for batch_images in batch_process(images, batch_size=100):
    feature_map = convnet.conv_forward(batch_images, 'same')
    print("Processed batch feature map shape:", feature_map.shape)
    # Continue with MaxPooling, Flatten, and Neural Network layers...
class MaxPooling:
    def __init__(self, f_map, pool_h=2, pool_w=2, stride=2):
        self.pool_filter_h = pool_h
        self.pool_filter_w = pool_w
        self.stride = stride

        self.examples = f_map.shape[0]
        self.fm_height = f_map.shape[1]
        self.fm_width = f_map.shape[2]
        self.channels = f_map.shape[3]
        self.last_input = f_map

    def max_forward(self):
        height_out = (self.fm_height - self.pool_filter_h) // self.stride + 1
        width_out = (self.fm_width - self.pool_filter_w) // self.stride + 1
        self.pool_output = np.zeros((self.examples, height_out, width_out, self.channels))

        for n in range(self.examples):
            for c in range(self.channels):
                for i in range(height_out):
                    for j in range(width_out):
                        row_start = i * self.stride
                        col_start = j * self.stride
                        window = self.last_input[n, row_start:row_start + self.pool_filter_h, col_start:col_start + self.pool_filter_w, c]
                        self.pool_output[n, i, j, c] = np.max(window)

        return self.pool_output


def flatten_layer(f_map):
    flat_output = f_map.reshape(f_map.shape[0], -1)
    return flat_output


class NeuralNetwork:
    def __init__(self, input_size, num_classes, hidden_units=100, learning_rate=0.01):
        self.learning_rate = learning_rate

        # Initialize weights
        self.weights1 = np.random.randn(input_size, hidden_units) * 0.01
        self.bias1 = np.zeros((1, hidden_units))
        self.weights2 = np.random.randn(hidden_units, num_classes) * 0.01
        self.bias2 = np.zeros((1, num_classes))

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def softmax(self, Z):
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)

    def forward(self, X):
        self.Z1 = np.dot(X, self.weights1) + self.bias1
        self.A1 = self.sigmoid(self.Z1)
        self.Z2 = np.dot(self.A1, self.weights2) + self.bias2
        self.A2 = self.softmax(self.Z2)
        return self.A2

    def compute_loss(self, Y, Y_hat):
        m = Y.shape[0]
        log_probs = -np.log(Y_hat[range(m), Y])
        loss = np.sum(log_probs) / m
        return loss

    def backprop(self, X, Y, Y_hat):
        m = X.shape[0]
        
        # Gradient of loss w.r.t. output
        dZ2 = Y_hat
        dZ2[range(m), Y] -= 1
        dW2 = np.dot(self.A1.T, dZ2) / m
        dB2 = np.sum(dZ2, axis=0, keepdims=True) / m
        
        dA1 = np.dot(dZ2, self.weights2.T)
        dZ1 = dA1 * self.A1 * (1 - self.A1)  # Sigmoid derivative
        dW1 = np.dot(X.T, dZ1) / m
        dB1 = np.sum(dZ1, axis=0, keepdims=True) / m
        
        # Update weights and biases
        self.weights1 -= self.learning_rate * dW1
        self.bias1 -= self.learning_rate * dB1
        self.weights2 -= self.learning_rate * dW2
        self.bias2 -= self.learning_rate * dB2


# Assuming you have labels `y` that are integer class labels
nn = NeuralNetwork(flattened_output.shape[1], num_classes=10)  # num_classes depends on your dataset
predictions = nn.forward(flattened_output)
loss = nn.compute_loss(y, predictions)
nn.backprop(flattened_output, y, predictions)

# After the neural network output is computed
predicted_classes = np.argmax(network_output, axis=1)
print("Predicted classes:", predicted_classes)


Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
Processed batch feature map shape: (100, 128, 128, 8)
